In [1]:
import pandas as pd 
import seaborn as sns 

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report


In [2]:
df = sns.load_dataset("titanic")

df = df[["survived", "pclass", "sex", "age", "fare", "embarked"]]
df = df.dropna()

In [3]:
df = pd.get_dummies(df, columns=["sex", "embarked"], drop_first=True)

x = df.drop("survived", axis=1)
y = df["survived"]

In [4]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [5]:
scaler = StandardScaler()

num_cols = ["age", "fare"]

x_train[num_cols] = scaler.fit_transform(x_train[num_cols]) 
x_test[num_cols] = scaler.transform(x_test[num_cols])

# Baseline Model tanlab shu bilan train qilamiz 

In [6]:
model = LogisticRegression()

model.fit(x_train, y_train)

y_pred = model.predict(x_test)

In [7]:
print("Accuracy natijasi:", accuracy_score(y_test, y_pred))
print("\n Classificdation report: \n")
print(classification_report(y_test, y_pred))

Accuracy natijasi: 0.7972027972027972

 Classificdation report: 

              precision    recall  f1-score   support

           0       0.76      0.93      0.84        80
           1       0.87      0.63      0.73        63

    accuracy                           0.80       143
   macro avg       0.82      0.78      0.79       143
weighted avg       0.81      0.80      0.79       143



# Bu Baseline Model hisoblanadi yani bunda quyidagilar yo`q 

- Hech qanday hyperparametr yo`q 
- hech qanday murakkab model yo`q 

# Hyperparametr Tuning 

# Hyperparameter - bu algaritmning settingslari hisoblanadi, agarda biz hyperparameter bermasak  u holda algaritm o`zi default parametrlar bilan ishlashni davom qildiradi

# Decision tree misolida aytsak masalan: max_depth = 1 bersak u xolda model underfittinga moyil bo`lib qoladi agrda shu qiymatni 100 bersak u xolda model overfittinga moyil bo`liob qoladi 

# manashu muammoni oldini olsih uchun biz eng optimal xolatni aniqlashimiz kerak


# parametr va hyperparametr nima? 

# Parametr bu -  
- internal coeffitsient 
- datalardan avtomatik o`rganadfi va manual kerak emas 
- barcha ichki settingslar bu parametr xisoblanadi

# Hyperparametr
- configurations (o`zgaruvchilar)
- trainingdan oldin qoyiladi 
- training processniu nazorat qiladi
- ular datalardan o`rganmaydi 

# Hyperparameter tuning nima  - model uchun optimal bo`lgan (eng yaxshi natija beruvchi) hyperparametrlarni izlash

# Nima uchun hyperparatmetr tuning  muhim 
- underffitting va overfittingni oldini olish 
- modelning accuracy va rubostness nes baholarini yaxshilash 
- bias variance tradeoff ni kamaytirish 
- vaqtni va resusrni tejash 

# Hyperparametr tuning dan so`ng quyidagilar yaxshilanadi 
- aniqlik oshadi 
- erorrlar kamayadi 
- generalization oshadi 

# Hyperparameter Turlari 
- Manual Search 
- Grid Search 
- Random Search 
- Bayesian Optimization 
- Optuna 

# MANUAL SEARCH 

# FOYDALILIGI: 
-   kichik va sodda modellarda foydasi kam 
# ISHLATILISHI: 
- tez tajribalar, kam pareametrli oddiy modellarda asosan ishlatiladi
# Misol : max_depth = 3 yoki max_depth = 5 qiymatini Decision tree unchun sinab ko`rish 

# Afzalligi: oddiy, kutubxonaqlar kerak emas

# kamchiligi : samarasiz, eng yaxshi parametrni o`tkazib yuborishi mumkin